In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 971.5 kB/s eta 0:00:00


In [11]:
# ✅ Auto-install dependencies (run once)
!pip install qiskit pylatexenc --quiet

# --- Imports ---
import math
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

# --- QFT circuit ---
def qft_circuit(n):
    qc = QuantumCircuit(n)
    for j in range(n):
        for k in range(j):
            angle = math.pi / (2 ** (j - k))
            qc.cp(angle, k, j)
        qc.h(j)
    # Swap qubits
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    qc.name = f"QFT_{n}"
    return qc

# --- Inverse QFT circuit ---
def qft_dagger_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    for j in reversed(range(n)):
        qc.h(j)
        for k in reversed(range(j)):
            angle = -math.pi / (2 ** (j - k))
            qc.cp(angle, k, j)
    qc.name = f"QFT†_{n}"
    return qc

# --- Helper: safe draw (use text if mpl unavailable) ---
def safe_draw(qc):
    try:
        return qc.draw('mpl')
    except Exception:
        return qc.draw('text')

# --- Show QFT and inverse QFT for 2, 3, 4 qubits ---
for n in [2, 3, 4]:
    print(f"\n==== QFT circuit for n = {n} qubits ====")
    display(safe_draw(qft_circuit(n)))
    print(f"\n==== Inverse QFT circuit for n = {n} qubits ====")
    display(safe_draw(qft_dagger_circuit(n)))

# --- Integration: simple phase estimation with QFT ---
def phase_estimation_example():
    n_count = 3   # counting qubits
    qc = QuantumCircuit(n_count + 1, n_count)
    # target qubit in |1>
    qc.x(n_count)
    # Hadamard on counting qubits
    qc.h(range(n_count))
    # Controlled-U (use phase gate)
    for q in range(n_count):
        qc.cp(2 * math.pi / (2 ** (q + 1)), q, n_count)
    # Apply inverse QFT
    inv_qft = qft_dagger_circuit(n_count)
    qc.compose(inv_qft, range(n_count), inplace=True)
    qc.measure(range(n_count), range(n_count))
    return qc

qc_phase = phase_estimation_example()
print("\n==== Phase Estimation Circuit ====")
display(safe_draw(qc_phase))

# --- Simulate and show results ---
sim = AerSimulator()
result = sim.run(qc_phase).result()
counts = result.get_counts()
plot_histogram(counts)
plt.show()



==== QFT circuit for n = 2 qubits ====


┌───┐                 
q_0: ┤ H ├─■─────────────X─
     └───┘ │P(π/2) ┌───┐ │ 
q_1: ──────■───────┤ H ├─X─
                   └───┘


==== Inverse QFT circuit for n = 2 qubits ====


┌───┐
q_0: ─X───────■────────┤ H ├
      │ ┌───┐ │P(-π/2) └───┘
q_1: ─X─┤ H ├─■─────────────
        └───┘


==== QFT circuit for n = 3 qubits ====


...

┌───┐                                                                     »
q_0: ┤ H ├─■─────────────■─────────────────■───────────────────────────────────»
     └───┘ │P(π/2) ┌───┐ │                 │                                   »
q_1: ──────■───────┤ H ├─┼────────■────────┼─────────────■──────────────────X──»
                   └───┘ │P(π/4)  │P(π/2)  │       ┌───┐ │                  │  »
q_2: ────────────────────■────────■────────┼───────┤ H ├─┼────────■─────────X──»
                                           │P(π/8) └───┘ │P(π/4)  │P(π/2) ┌───┐»
q_3: ──────────────────────────────────────■─────────────■────────■───────┤ H ├»
                                                                          └───┘»
«        
«q_0: ─X─
«      │ 
«q_1: ─┼─
«      │ 
«q_2: ─┼─
«      │ 
«q_3: ─X─
«


==== Inverse QFT circuit for n = 4 qubits ====


»
q_0: ─X────────────────────────────────■───────────────────■─────────────»
      │                                │                   │        ┌───┐»
q_1: ─┼───X─────────────■──────────────┼─────────■─────────┼────────┤ H ├»
      │   │             │        ┌───┐ │         │P(-π/2)  │P(-π/4) └───┘»
q_2: ─┼───X───■─────────┼────────┤ H ├─┼─────────■─────────■─────────────»
      │ ┌───┐ │P(-π/2)  │P(-π/4) └───┘ │P(-π/8)                          »
q_3: ─X─┤ H ├─■─────────■──────────────■─────────────────────────────────»
        └───┘                                                            »
«               ┌───┐
«q_0: ─■────────┤ H ├
«      │P(-π/2) └───┘
«q_1: ─■─────────────
«                    
«q_2: ───────────────
«                    
«q_3: ───────────────
«


==== Phase Estimation Circuit ====


┌───┐                                                                    »
q_0: ┤ H ├─■────────────────────────X─────────────────■──────────────■────────»
     ├───┤ │                        │                 │        ┌───┐ │P(-π/2) »
q_1: ┤ H ├─┼──────■─────────────────┼───────■─────────┼────────┤ H ├─■────────»
     ├───┤ │      │                 │ ┌───┐ │P(-π/2)  │P(-π/4) └┬─┬┘          »
q_2: ┤ H ├─┼──────┼────────■────────X─┤ H ├─■─────────■─────────┤M├───────────»
     ├───┤ │P(π)  │P(π/2)  │P(π/4)    └───┘                     └╥┘           »
q_3: ┤ X ├─■──────■────────■─────────────────────────────────────╫────────────»
     └───┘                                                       ║            »
c: 3/════════════════════════════════════════════════════════════╩════════════»
                                                                 2            »
«     ┌───┐┌─┐
«q_0: ┤ H ├┤M├
«     └┬─┬┘└╥┘
«q_1: ─┤M├──╫─
«      └╥┘  ║ 
«q_2: ──╫───╫─
«       ║   ║ 
«q_3: ──╫───╫─
«       ║   ║ 
«c: 3/══╩═══╩═
«       1   0